In [1]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
import numpy as np
from PIL import Image
import gc, math
import pickle

from keras.models import Sequential
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.models import model_from_json
from keras.models import Model
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras import backend as K
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import log_loss, accuracy_score, confusion_matrix


from cnnmodels import vgg_std16_model, preprocess_input, create_rect5, load_img, train_generator, test_generator
from cnnmodels import identity_block, testcv_generator, conv_block, resnet50_model

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
# Functions
def preprocess_input(x):
    #resnet50 image preprocessing
    # 'RGB'->'BGR'
    x = x[:, :, ::-1]
    x[:, :, 0] -= 100
    x[:, :, 1] -= 115
    x[:, :, 2] -= 124
    return x

def create_rect5(row):
    if is_seal:
        return plt.Rectangle((row['x0'], row['y0']), row['w'], row['h'], color='red', fill=False, lw=2)
    else:
        return plt.Rectangle((row['x0'], row['y0']), row['w'], row['h'], color='red', fill=False, lw=4)

def load_img(path, bbox, target_size=None):
    img = Image.open(path)
    img = img.convert('RGB')
    cropped = img.crop((bbox[0],bbox[1],bbox[2],bbox[3]))
    if target_size:
        cropped = cropped.resize((target_size[1], target_size[0]))
    return cropped

In [3]:
# Params
img_rows, img_cols = 224, 224 # Resolution of inputs
channel = 3
num_class = 2
ROWS, COLS = 224, 224
BATCHSIZE = 64 # 128
SEAL_CLASSES = ['seals_0', 'seals_1', 'seals_2_4', 'seals_5_10', 'seals_11+']
nb_perClass = int(BATCHSIZE / len(SEAL_CLASSES))
TRAIN_DIR = '../darknet/seals/JPEGImagesBlk'
TEST_DIR = '../darknet/seals/JPEGImagesTest'
num_class = len(SEAL_CLASSES)

In [4]:
# CV object detections
rfcnCVtmp = pd.read_pickle('../coords/rfcnmultiCV.pkl')
# Read in the previous preds 
dftmp = pd.concat([pd.read_csv('../coords/vggCVPreds2604_fold2.csv'),
                pd.read_csv('../coords/vggCVPreds2604_fold1.csv')])
dftmp.columns = ['img1', 'predSeal']
rfcnCV = pd.concat([rfcnCVtmp.reset_index(drop=True), dftmp.reset_index(drop=True)], axis=1)
rfcnCV = rfcnCV[rfcnCV['predSeal']>0.2].reset_index(drop=True)
rfcnCV.head()

,img,proba,x0,y0,x1,y1,seal,img1,predSeal
0,332_22,0.998,444.00,9.25,544.00,109.25,1,332_22,0.987439
1,332_22,0.925,261.45,195.15,361.45,295.15,1,332_22,0.999877
2,448_21,0.989,0.00,422.25,100.00,522.25,0,448_21,0.942756
3,338_32,0.958,444.00,444.00,544.00,544.00,2,338_32,0.944517
4,338_32,0.917,444.00,4.40,544.00,104.40,2,338_32,0.994015


In [5]:
# Add on the classes for bins
rfcnCV['seal_cut'] = pd.cut(rfcnCV['seal'], bins = [-1,0,1,4,10,30])
rfcnCV['seals_0'] = np.where(rfcnCV['seal']==0, 1, 0)
rfcnCV['seals_1'] = np.where(rfcnCV['seal'] == 1, 1, 0)
rfcnCV['seals_2_4'] = np.where(rfcnCV['seal'].between(2,4), 1, 0)
rfcnCV['seals_5_10'] = np.where(rfcnCV['seal'].between(5,10), 1, 0)
rfcnCV['seals_11+'] = np.where(rfcnCV['seal']>10, 1, 0)
rfcnCV.head(5)

,img,proba,x0,y0,x1,y1,seal,img1,predSeal,seal_cut,seals_0,seals_1,seals_2_4,seals_5_10,seals_11+
0,332_22,0.998,444.00,9.25,544.00,109.25,1,332_22,0.987439,"(0, 1]",0,1,0,0,0
1,332_22,0.925,261.45,195.15,361.45,295.15,1,332_22,0.999877,"(0, 1]",0,1,0,0,0
2,448_21,0.989,0.00,422.25,100.00,522.25,0,448_21,0.942756,"(-1, 0]",1,0,0,0,0
3,338_32,0.958,444.00,444.00,544.00,544.00,2,338_32,0.944517,"(1, 4]",0,0,1,0,0
4,338_32,0.917,444.00,4.40,544.00,104.40,2,338_32,0.994015,"(1, 4]",0,0,1,0,0


In [6]:
# Test object detections
rfcnTsttmp = pd.read_pickle('../coords/rfcnmultiTst.pkl')
# Read in the previous preds 
dftmp = pd.read_csv('../coords/vggTestPreds2604.csv')
dftmp.columns = ['img1', 'predSeal']
rfcnTst = pd.concat([rfcnTsttmp.reset_index(drop=True), dftmp.reset_index(drop=True)], axis=1)
rfcnTst = rfcnTst[rfcnTst['predSeal']>0.2].reset_index(drop=True)
print(rfcnTst.shape)
rfcnTst.head()

(863969, 8)


,img,proba,x0,y0,x1,y1,img1,predSeal
0,11710_62,0.963,444.00,12.00,544.00,112.00,11710_62,0.741276
1,17920_13,0.939,3.85,444.00,103.85,544.00,17920_13,0.788811
2,7121_51,0.989,55.25,424.95,155.25,524.95,7121_51,0.606451
3,15778_83,0.982,0.00,444.00,100.00,544.00,15778_83,0.619216
4,15778_83,0.970,444.00,218.40,544.00,318.40,15778_83,0.873984


In [7]:
for c, row in rfcnCV.iterrows():
    if c==5:break
    print row.tolist()[10:]

[0, 1, 0, 0, 0]
[0, 1, 0, 0, 0]
[1, 0, 0, 0, 0]
[0, 0, 1, 0, 0]
[0, 0, 1, 0, 0]


In [8]:
def train_generator(datagen, df):
    while 1:
        batch_x = np.zeros((BATCHSIZE, ROWS, COLS, 3), dtype=K.floatx())
        batch_y = np.zeros((BATCHSIZE, len(SEAL_CLASSES)), dtype=K.floatx())
        fn = lambda obj: obj.loc[np.random.choice(obj.index, size=nb_perClass, replace=False),:]
        batch_df = df.groupby(['seal_cut'], as_index=True).apply(fn)
        i = 0
        for index,row in batch_df.iterrows():
            row = row.tolist()
            image_file = os.path.join(TRAIN_DIR, row[0])
            seal = row[6]
            bbox = row[2:6]
            cropped = load_img(image_file+'.jpg',bbox,target_size=(ROWS,COLS))
            x = np.asarray(cropped, dtype=K.floatx())
            x = datagen.random_transform(x)
            x = preprocess_input(x)
            batch_x[i] = x
            batch_y[i] = row[10:] # Add in all classes
            i += 1
        yield (batch_x.transpose(0, 3, 1, 2), batch_y)
        #return (batch_x.transpose(0, 3, 1, 2), batch_y)

def test_generator(df, datagen = None, batch_size = BATCHSIZE):
    n = df.shape[0]
    batch_index = 0
    while 1:
        current_index = batch_index * batch_size
        if n >= current_index + batch_size:
            current_batch_size = batch_size
            batch_index += 1    
        else:
            current_batch_size = n - current_index
            batch_index = 0        
        batch_df = df[current_index:current_index+current_batch_size]
        batch_x = np.zeros((batch_df.shape[0], ROWS, COLS, 3), dtype=K.floatx())
        i = 0
        for index,row in batch_df.iterrows():
            row = row.tolist()
            image_file = os.path.join(TEST_DIR, row[0]+'.jpg')
            bbox = row[2:6]
            cropped = load_img(image_file,bbox,target_size=(ROWS,COLS))
            x = np.asarray(cropped, dtype=K.floatx())
            if datagen is not None: x = datagen.random_transform(x)            
            x = preprocess_input(x)
            batch_x[i] = x
            i += 1
        if batch_index%16000 == 0: print(batch_index)
        #return(batch_x.transpose(0, 3, 1, 2))
        yield(batch_x.transpose(0, 3, 1, 2))
        
# Data generator
train_datagen = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True)

In [9]:
# Lets make our validation set
CVsplit = rfcnCV.img.str.split('_').apply(lambda x: x[0]).astype(int) % 40 == 0
train_df = rfcnCV[~CVsplit]
valid_df = rfcnCV[CVsplit]
test_df = rfcnTst

In [10]:
test_df.head()

,img,proba,x0,y0,x1,y1,img1,predSeal
0,11710_62,0.963,444.00,12.00,544.00,112.00,11710_62,0.741276
1,17920_13,0.939,3.85,444.00,103.85,544.00,17920_13,0.788811
2,7121_51,0.989,55.25,424.95,155.25,524.95,7121_51,0.606451
3,15778_83,0.982,0.00,444.00,100.00,544.00,15778_83,0.619216
4,15778_83,0.970,444.00,218.40,544.00,318.40,15778_83,0.873984


In [11]:
# validation_data (valid_x,valid_y)
df_1 = valid_df
l = valid_df.groupby('seal_cut').size() 
nb_NoF_valid = math.ceil(l.sum()/10)
valid_x = np.zeros((valid_df.shape[0], ROWS, COLS, 3), dtype=K.floatx())
valid_y = np.zeros((valid_df.shape[0], len(SEAL_CLASSES)), dtype=K.floatx())
i = 0
for index,row in valid_df.iterrows():
    row = row.tolist()
    image_file = os.path.join(TRAIN_DIR, row[0])
    seal = row[6]
    bbox = row[2:6]
    cropped = load_img(image_file+'.jpg',bbox,target_size=(ROWS,COLS))
    x = np.asarray(cropped, dtype=K.floatx())
    x = preprocess_input(x)
    valid_x[i] = x
    valid_y[i] = row[10:]
    i += 1
valid_x = valid_x.transpose(0, 3, 1, 2)
valid_x.shape

(1862, 3, 224, 224)

In [12]:
# Load our model
nb_epoch = 2
samples_per_epoch = 70400
model = vgg_std16_model(ROWS, COLS, channel, num_class)

# Start Fine-tuning
model.fit_generator(train_generator(train_datagen, train_df),
          nb_epoch=nb_epoch,
          samples_per_epoch=samples_per_epoch, #50000,
          verbose=1,
          validation_data=(valid_x, valid_y),
          )

Epoch 1/2
 64/100 [==================>...........] - ETA: 2s - loss: 3.9028 - acc: 0.1875

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/engine/training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


128/100 [======================================] - 55s - loss: 2.9118 - acc: 0.2266 - val_loss: 1.5962 - val_acc: 0.2170
Epoch 2/2
128/100 [======================================] - 54s - loss: 1.5886 - acc: 0.2266 - val_loss: 1.9265 - val_acc: 0.0569


In [13]:
for layer in model.layers[10:]:
    layer.trainable = True
model.optimizer.lr = 1e-4
nb_epoch = 3
model.fit_generator(train_generator(train_datagen, df=train_df),
          nb_epoch=nb_epoch,
          samples_per_epoch=samples_per_epoch,
          verbose=1,
          validation_data=(valid_x, valid_y),
          )

Epoch 1/3
128/100 [======================================] - 55s - loss: 1.5332 - acc: 0.2656 - val_loss: 1.5132 - val_acc: 0.3330
Epoch 2/3
128/100 [======================================] - 54s - loss: 1.6941 - acc: 0.1875 - val_loss: 1.5036 - val_acc: 0.3539
Epoch 3/3
128/100 [======================================] - 54s - loss: 1.8403 - acc: 0.1875 - val_loss: 1.4509 - val_acc: 0.2664


In [16]:
# test_preds = test_model.predict_generator(test_generator(test_df), val_samples=test_df.shape[0]))
test_preds = model.predict_generator(test_generator(test_df), val_samples=test_df.shape[0])

0


In [17]:
df = pd.concat([test_df, pd.DataFrame(test_preds,  columns=SEAL_CLASSES)], axis=1)
df.to_pickle('../coords/vggTestMultiPreds1605.pkl')

In [27]:
df[[0]+range(8, 13)].to_csv('../coords/vggTestMultiPreds1605.csv', index=False)
df.head(5)

,img,proba,x0,y0,x1,y1,img1,predSeal,seals_0,seals_1,seals_2_4,seals_5_10,seals_11+
0,11710_62,0.963,444.00,12.00,544.00,112.00,11710_62,0.741276,0.129170,0.351934,0.211231,0.233157,0.074509
1,17920_13,0.939,3.85,444.00,103.85,544.00,17920_13,0.788811,0.104397,0.359540,0.297087,0.191007,0.047968
2,7121_51,0.989,55.25,424.95,155.25,524.95,7121_51,0.606451,0.136604,0.307602,0.190649,0.279917,0.085228
3,15778_83,0.982,0.00,444.00,100.00,544.00,15778_83,0.619216,0.127126,0.347254,0.289869,0.178432,0.057318
4,15778_83,0.970,444.00,218.40,544.00,318.40,15778_83,0.873984,0.165854,0.280094,0.265157,0.216112,0.072784
